# Measurement files XCT Loading

Notebook to load Measurement files ut data into the databse

## Imports

In [13]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load
from pathlib import Path
import tifffile as tiff
import os
from tqdm import tqdm

## Auxiliar functions

In [14]:
def read_sequence2(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.

    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted(
        [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if (f.endswith(".tiff") or f.endswith(".tif"))
        ]
    )

    # Get the total number of TIFF files
    total_files = len(tiff_files)

    # Read each TIFF file and update progress
    volume = []

    for i, file_path in tqdm(enumerate(tiff_files), total=total_files, desc="Reading TIFF files"):
        slice_data = tiff.imread(file_path)
        volume.append(slice_data)

    return np.array(volume)

## Connection

In [15]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## File loading

Here we have to load the file of the measurement to be saved in the database.

The parent file path has to be explained in depth: Measurements include more than one sample, but then the files are cropped so each sample can be treated individually. Parent file represents the file from where the one being saved is extracted from.

If the file being saved is not extracted from any other, set paren_file_path to None

In [17]:
parent_file_path = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Fabricacion Nacho\02_Laminados_Nacho\Probetas\Na_01\Na_01_bottom_2800x214x3942.tif')
file_path = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Fabricacion Nacho\02_Laminados_Nacho\Probetas\Na_01\Na_01_bottom_2800x214x3942_reslice_top.tif')

# Check if the path is a directory or a file
if file_path.is_dir():
    # If it's a directory, read all TIFF files in the directory
    file = read_sequence2(file_path)
else:
    # If it's a file, read the single TIFF file
    file = tiff.imread(file_path)

# Measurement type data

We have to set the measurment type id, in this case is the ultrasound 2024 JI methodology so the id is 1.

In case you are not sure of the mesaurement type id, you can check it in the measurement_type table in the next cell.

Remember that if there is a parent file, the measurement_type id will be forced to be the same as the paren measurement.

In [18]:
parent_table_name = 'measurementtypes'

parent_id_column = parent_table_name[:-1] + '_id'

parent_data = qrs.get_data(parent_table_name)

parent_data

,id_measurementtype,name_measurementtype,description_measurementtype
0,2,XCT_2024,"First XCT measurements, 2024 metodology."
1,3,UT_2024,"UT measurements, 2024 metodology. Used for the..."
2,4,UT_2025_1,"UT measurements, 2025 metodology. Measuring at..."


In [19]:
#we set the id
measurement_type_id = 2

## Measurement data

Now we have to set the attributes and metadata of the measurements:

1. Main attributes

    1. Sample_names: Its a list with the names of all the samples present in the file. If the names are not written as in the database it wont work so be precise.

    2. file_path: The file location

    3. Parent_file_id: In this variable save the path to the parent file. A parent file is the file where the file we want to save was precomputed from, for example the amplitude file is computed from the rf file. This will be automatically extracted later.

2. Metadata

    1. Dimensions: Dimensions are automatically computed from the loaded file

    2. Dtype: The precision type of the file

    3. File_type: The file extension, if its a folder of tiffs the type is folder
    

In [ ]:
height = file.shape[0]
width = file.shape[1]
depth = file.shape[2]
dtype = str(file.dtype)

file_type = file_path.suffix

signal_type  = 'Amplitude'

axes_order = ['z', 'y', 'x']

sample_names = ['prueba']

aligned = True

equalized = True

transformation = 'Frontal_90righted'

if parent_file_path is None:
    load.load_xct_measurement(conn, str(file_path),measurement_type_id,height,width,depth,dtype,file_type,sample_names,aligned,equalized,axes_order,None)
else:
    load.load_xct_measurement(conn, str(file_path),measurement_type_id,height,width,depth,dtype,file_type,sample_names,aligned,equalized,axes_order,str(parent_file_path),transformation)

XCT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Fabricacion Nacho\02_Laminados_Nacho\Probetas\Na_01\Na_01_bottom_2800x214x3942_reslice_top.tif' loaded with ID: 97


## Make sure insert was correct

In [22]:
data = qrs.relation_metadata('measurements','samples','sample_measurements')

data

,id_measurement,file_path_measurement,parent_measurement_id_measurement,measurementtype_id_measurement,height_measurement,width_measurement,depth_measurement,dtype_measurement,file_type_measurement,signal_type_measurement,...,transformations_measurement,id_sample,name_sample,panel_id_sample,geometry_type_sample,height_sample,width_sample,thickness_sample,keyhole_sample,parallel_faces_sample
0,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,3,505 cardinal,203 cardinal,372 cardinal,uint8 nominal,tif nominal,RF nominal,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
1,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3,505 cardinal,120 cardinal,45 cardinal,uint8 nominal,tif nominal,RF nominal,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
2,21,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,2,3737 cardinal,1770 cardinal,1015 cardinal,uint16 nominal,folder nominal,NaN,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
3,22,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,21.0,2,3224 cardinal,1770 cardinal,1015 cardinal,uint8 nominal,folder nominal,NaN,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
4,26,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,22.0,2,3224 cardinal,1610 cardinal,240 cardinal,uint8 nominal,folder nominal,NaN,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
5,41,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,26.0,2,225 cardinal,3279 cardinal,1542 cardinal,uint8 nominal,folder nominal,NaN,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
6,43,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,4,448 cardinal,190 cardinal,492 cardinal,uint8 nominal,tif nominal,RF nominal,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
7,44,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,43.0,4,448 cardinal,125 cardinal,48 cardinal,uint8 nominal,tif nominal,RF nominal,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
8,60,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,26.0,2,3224 cardinal,1610 cardinal,240 cardinal,uint8 nominal,folder nominal,NaN,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
9,84,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,60.0,2,203 cardinal,3283 cardinal,1535 cardinal,uint8 nominal,folder nominal,NaN,...,NaN,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm,True Boolean,NaN
